### 1. Google Drive 마운트 및 라이브러리 임포트


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard ### <--- TensorBoard 콜백 임포트

import numpy as np
import pandas as pd
import glob
import os
import datetime ### <--- 로그 디렉토리 생성을 위한 datetime 임포트
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

### 2. 데이터 로드 및 전처리
지정한 디렉터리 구조를 탐색하여 모든 CSV 파일을 읽고, 시계열 데이터를 모델 학습에 적합한 형태(시계열 윈도우)로 가공합니다.

- SEQUENCE_LENGTH: 모델이 한 번에 입력으로 받을 시간 스텝의 길이입니다. 이 값을 조절하여 성능을 튜닝할 수 있습니다.
- STEP: 윈도우를 얼마만큼씩 이동하며 만들지 결정합니다. 1이면 한 스텝씩 이동합니다.    

In [ ]:
# 데이터 경로 설정 (본인의 구글 드라이브 경로에 맞게 수정하세요)
BASE_PATH = '/content/drive/MyDrive/낙상'

# 시계열 윈도우 생성을 위한 하이퍼파라미터
SEQUENCE_LENGTH = 100  # 100개의 타임스텝을 하나의 시퀀스로 사용
STEP = 1               # 윈도우를 한 스텝씩 이동

def load_and_preprocess_data(base_path):
    """
    지정된 경로에서 모든 CSV 파일을 로드하고, 시계열 윈도우로 변환합니다.
    """
    all_files = glob.glob(os.path.join(base_path, 'case*', '*', '*.csv'), recursive=True)
    print(f"총 {len(all_files)}개의 파일을 찾았습니다.")

    sequences = []
    labels = []

    for file_path in all_files:
        try:
            # CSV 파일 로드 (첫 번째 행을 헤더로 사용하지 않음)
            df = pd.read_csv(file_path, header=None)
            
            # 컬럼명 설정: timestamp, label, feature_0, ..., feature_255
            columns = ['timestamp', 'label'] + [f'feature_{i}' for i in range(256)]
            df.columns = columns
            
            # feature 데이터와 label 분리
            features = df.drop(['timestamp', 'label'], axis=1).values
            label_data = df['label'].values

            # 데이터 정규화 (StandardScaler 사용)
            scaler = StandardScaler()
            features_scaled = scaler.fit_transform(features)

            # 시계열 윈도우 생성
            for i in range(0, len(df) - SEQUENCE_LENGTH, STEP):
                sequences.append(features_scaled[i: i + SEQUENCE_LENGTH])
                
                # 윈도우 내에 낙상(1)이 한 번이라도 포함되면 해당 윈도우의 레이블을 1로 지정
                window_labels = label_data[i: i + SEQUENCE_LENGTH]
                if 1 in window_labels:
                    labels.append(1)
                else:
                    labels.append(0)

        except Exception as e:
            print(f"파일 처리 중 오류 발생: {file_path}, 오류: {e}")

    return np.array(sequences), np.array(labels)

# 데이터 로드 및 전처리 실행
X, y = load_and_preprocess_data(BASE_PATH)

print(f"\n생성된 총 시퀀스 수: {X.shape[0]}")
print(f"시퀀스 형태 (데이터 수, 시퀀스 길이, 특징 수): {X.shape}")
print(f"레이블 형태: {y.shape}")
print(f"낙상(1) 데이터 수: {np.sum(y)} / 비낙상(0) 데이터 수: {len(y) - np.sum(y)}")


# 학습 데이터와 검증 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("\n--- 데이터 분할 결과 ---")
print(f"학습 데이터 형태: {X_train.shape}")
print(f"검증 데이터 형태: {X_val.shape}")

### 3. CNN-LSTM 모델 구축
CNN으로 시계열 데이터의 지역적 특징을 추출하고, LSTM으로 시간적 패턴을 학습하는 결합 모델을 정의합니다.

In [ ]:
def build_cnn_lstm_model(input_shape):
    """
    CNN-LSTM 결합 모델을 생성합니다.
    """
    model = Sequential()
    
    # CNN 계층: 시계열의 지역적 특징 추출
    # TimeDistributed는 각 타임스텝에 동일한 Conv1D를 적용하기 위함이지만,
    # 여기서는 전체 시퀀스에서 특징을 뽑는 Conv1D를 먼저 적용합니다.
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.5))

    # LSTM 계층: 추출된 특징의 시간적 순서 학습
    model.add(LSTM(100, return_sequences=False)) # 마지막 출력만 사용
    model.add(Dropout(0.5))
    
    # 완전 연결 계층 (분류기)
    model.add(Dense(100, activation='relu'))
    model.add(Dense(1, activation='sigmoid')) # 이진 분류를 위한 시그모이드 활성화 함수
    
    return model

# 모델 생성
input_shape = (SEQUENCE_LENGTH, X_train.shape[2]) # (시퀀스 길이, 특징 수)
model = build_cnn_lstm_model(input_shape)

# 모델 컴파일
model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

# 모델 구조 출력
model.summary()

### 4. TensorBoard 실행 및 모델 학습 (핵심 수정 부분)
이 부분이 TensorBoard를 연동하는 핵심입니다.

1. 실행할 때마다 로그를 구분하기 위해 현재 시간을 이용해 로그 디렉터리 경로를 만듭니다.
2. Colab의 매직 커맨드 %load_ext tensorboard와 %tensorboard --logdir logs/fit을 사용해 TensorBoard를 실행합니다. 이 코드를 model.fit 보다 먼저 실행하면, 학습이 시작됨과 동시에 모니터링 UI가 나타납니다.
3. model.fit의 callbacks 리스트에 TensorBoard 콜백을 추가합니다.

In [ ]:
### --- TensorBoard 연동 부분 --- ###

# 1. TensorBoard 로그를 저장할 고유한 디렉토리 설정
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# 2. TensorBoard 콜백 생성
# histogram_freq=1은 매 에포크마다 가중치와 편향의 분포를 시각화합니다.
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# 3. Colab에서 TensorBoard 확장 프로그램 로드 및 실행
%load_ext tensorboard
%tensorboard --logdir logs/fit

### --------------------------- ###


# 조기 종료 콜백 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


# 모델 학습 (callbacks 리스트에 tensorboard_callback 추가)
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=64,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping, tensorboard_callback] ### <--- 수정된 부분
)

print("\n모델 학습이 완료되었습니다.")